### Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [16]:
# Import necessary dependencites
from collections import Counter, defaultdict
from copy import deepcopy
import pandas as pd
import random
import re
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split

<a id="ngram-dataset"></a>
### Which ngram dataset to use?
I decided to use the provided fivegram dataset do to its relatively large size and a "room" to play. From my perspective, five words IS better than three words.

### What is the main idea for my version of solving the given task?
It is simple:
- Create a basic Norvig's Corrector implementation - A good baseline for us;
- Modify the basic Norvig's Corrector by adding N-Gram model inside;
- Compare the results;
- Think of possible improvements for my solution.

### What kind of modifications were made to improve basic Norvig's Corrector implementation?
I decided to create a resulting probability for each candidate by summing up two calculated probabilities:
- Norvig's Corrector probability;
- N-Gram model probability;
- Many more discussed at the end of this notebook.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

#### Norvig's Corrector implementation

We will use provided fivegram dataset for evaluation.

In [2]:
# Read the main file and store lines as lists in list
text_storage = []

with open('/home/viper/Data_science/NLP/assignments/assignment_1/useful_data/fivegrams.txt', 'r') as f:
    for string in f.readlines():
        text_storage.append(list(map(lambda x: x.lower(), string.split()[1:])))

In [3]:
# Split text storage into training and validation sets
train_storage, val_storage = train_test_split(text_storage, test_size=0.1)
del text_storage

In [4]:
# Function to calculate the amount of each word in text
def calculate_amount(text: list[list[str]]) -> Counter:
    counter = Counter()
    for inner_list in text:
        counter.update(inner_list)
    
    return counter

In [5]:
train_counter = calculate_amount(train_storage)
val_counter = calculate_amount(val_storage)

In [6]:
train_counter.most_common()[:10]

[('the', 400974),
 ('to', 260703),
 ('of', 241476),
 ('a', 158344),
 ('in', 113456),
 ('and', 98590),
 ('that', 89322),
 ('i', 65972),
 ("n't", 65604),
 ('it', 62704)]

In [7]:
val_counter.most_common()[:10]

[('the', 44762),
 ('to', 28954),
 ('of', 27009),
 ('a', 17555),
 ('in', 12657),
 ('and', 10942),
 ('that', 9796),
 ('i', 7335),
 ("n't", 7294),
 ('it', 6997)]

In [ ]:
# Implement Norvig's spelling corrector
class NorvigCorrector():
    def __init__(self, train_dict: Counter) -> None:
        """
        Initialize the NorvigCorrector class.
        Implementation taken from: https://norvig.com/spell-correct.html

        Parameters
        ----------
        train_dict: Counter:
            Counter object that contains the amount of each word that is encountered in training text.
            For example:
            ('the', 401257),
            ('to', 260604),
            ('of', 241419),
            ('a', 158322),
            ('in', 113517),
            etc.
        
        Returns
        ----------
        NorvigCorrector class object.
        """

        self.train_dict = train_dict
        self.N = sum(train_dict.values())
    
    def P(self, word: str, N: int=None) -> float:
        """
        Calculate probability of given word.
        """

        if N is None:
            N = self.N
        
        return self.train_dict[word] / N
    
    def correction(self, word: str) -> str:
        """
        Find most probable spelling correction for given word.
        """

        return max(self.candidates(word), key=self.P)
    
    def candidates(self, word: str) -> set:
        """
        Generate possible spelling corrections for given word.
        """

        return (self.known([word]) or (self.known(self.edits1(word)))
                or self.known(self.edits2(word)) or [word])
    
    def known(self, words: str | list[str]) -> set:
        """
        Find the subset of words that appear in the dictionary of `NorvigCorrector.train_words`.
        """

        return set(w for w in words if w in self.train_dict.keys())
    
    def edits1(self, word: str) -> set:
        """
        Generate all possible edits that are one edit away from given word.
        """

        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]

        return set(deletes + transposes + replaces + inserts)
    
    def edits2(self, word: str) -> set:
        """
        Generate all possible edits that are two edits away from given word.
        """

        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

In [359]:
norvig_corrector = NorvigCorrector(train_counter)

In [360]:
# Test Norvig Corrector
norvig_corrector.correction('eing')

'being'

### Generate a proper validation set for our models

The core idea is the following:
- Take each word in `val_storage` that has at least one neighboring word from each side (so, we take only 3 words from each list inside `val_storage`);
- Modify one taken word once or twice (modifications are chosen randomly).

In [13]:
val_storage[:10]

[['that', 'might', 'be', 'interpreted', 'as'],
 ['made', 'a', 'lot', 'of', 'sacrifices'],
 ['plant', 'biological', 'and', 'molecular', 'processes'],
 ['no', 'reason', 'why', 'they', 'should'],
 ['and', 'he', 'told', 'her', 'he'],
 ['out', 'of', 'the', 'sight', 'of'],
 ['i', 'knew', 'how', 'to', 'work'],
 ['up', 'out', 'of', 'the', 'mud'],
 ['then', 'he', 'is', 'going', 'to'],
 ['of', 'the', 'small', 'sample', 'size']]

In [15]:
len(train_storage)

939841

In [314]:
def random_delete(word: str) -> str:
    """
    Randomly delete one character from the given word.
    """

    if len(word) <= 2:
        selected_char = 0
    else:
        selected_char = random.randint(0, len(word) - 1)

    return word[:selected_char] + word[selected_char + 1:]

In [315]:
def random_transpose(word: str) -> str:
    """
    Randomly change two adjacent characters in the given word.
    """

    if len(word) <= 1:
        return word

    if len(word) == 2:
        selected_char = 0
    else:
        selected_char = random.randint(1, len(word) - 1)

    return word[:selected_char - 1] + word[selected_char] + word[selected_char - 1] + word[selected_char + 1:]

In [316]:
def random_replace(word: str) -> str:
    """
    Randomly replace one character with random letter in the given word.
    """

    if len(word) <= 1:
        selected_char = 0
    else:
        selected_char = random.randint(0, len(word) - 1)
    letters = 'abcdefghijklmnopqrstuvwxyz'

    return word[:selected_char] + random.choice(letters) + word[selected_char + 1:]

In [317]:
def random_insert(word: str) -> str:
    """
    Randomly insert one character with random letter in the given word.
    """

    if len(word) <= 1:
        selected_char = 0
    else:
        selected_char = random.randint(0, len(word) - 1)
    letters = 'abcdefghijklmnopqrstuvwxyz'

    return word[:selected_char] + random.choice(letters) + word[selected_char:]

In [318]:
random_insert('happy')

'happfy'

In [319]:
random_replace('happy')

'haupy'

In [320]:
random_transpose('happy')

'hapyp'

In [321]:
random_delete('happy')

'hapy'

In [322]:
def random_modification(word: str) -> str:
    """
    Make one or two random modifications for a word.
    """

    modifications = {
        0: lambda x: random_insert(x),
        1: lambda x: random_replace(x),
        2: lambda x: random_transpose(x),
        3: lambda x: random_delete(x),
    }

    modifications_count = random.randint(1, 2)

    if modifications_count == 2:
        return modifications.get(random.randint(0, 3))(modifications.get(random.randint(0, 3))(word))
    else:
        return modifications.get(random.randint(0, 3))(word)

In [323]:
random_modification('happy')

'happy'

In [ ]:
def modify_list(word_list: list[str]) -> list[str]:
    """
    Modify one word from given list of words.
    """

    assert(len(word_list) == 5), (
        "The length of given `word_list` must equal to 5."
    )

    chosen_word_idx = 2

    if len(word_list[1]) < 3 and len(word_list[2]) < 3 and len(word_list[3]) < 3:
        return word_list
    
    modified_word = random_modification(word_list[chosen_word_idx])
    
    return (word_list[:chosen_word_idx] + [modified_word] + word_list[chosen_word_idx + 1:], chosen_word_idx, word_list)

In [326]:
modify_list(['will', 'have', 'significant', 'effects', 'on'])

(['will', 'have', 'simnificant', 'effects', 'on'],
 2,
 ['will', 'have', 'significant', 'effects', 'on'])

In [ ]:
# Generate validation set using created functions
def generate_modified_set(sent_list: list[list[str]]) -> None:
    """
    Generate the modified lists of given sentences.
    """

    modified_sentences = []
    modified_word_idxs = []

    for sent in sent_list:
        res = modify_list(sent)
        modified_sentences.append(res[0])
        modified_word_idxs.append(res[1])
    
    return pd.DataFrame(
        data={
            "modified_sentence": modified_sentences,
            "modified_word_idxs": modified_word_idxs
        }
    )

In [328]:
val_storage

[['that', 'might', 'be', 'interpreted', 'as'],
 ['made', 'a', 'lot', 'of', 'sacrifices'],
 ['plant', 'biological', 'and', 'molecular', 'processes'],
 ['no', 'reason', 'why', 'they', 'should'],
 ['and', 'he', 'told', 'her', 'he'],
 ['out', 'of', 'the', 'sight', 'of'],
 ['i', 'knew', 'how', 'to', 'work'],
 ['up', 'out', 'of', 'the', 'mud'],
 ['then', 'he', 'is', 'going', 'to'],
 ['of', 'the', 'small', 'sample', 'size'],
 ['deal', 'with', 'them', 'in', 'a'],
 ['do', "n't", 'quite', 'understand', 'the'],
 ['that', 'it', 'was', 'a', 'simple'],
 ['the', 'one', 'who', 'had', 'to'],
 ['of', 'the', 'underlying', 'causes', 'of'],
 ['to', 'do', 'is', 'to', 'write'],
 ['i', 'consider', 'them', 'to', 'be'],
 ['volume', 'and', 'complexity', 'of', 'the'],
 ['now', 'we', 'are', 'on', 'the'],
 ['about', 'the', 'long-term', 'health', 'of'],
 ['that', 'seems', 'to', 'be', 'most'],
 ['i', 'had', 'never', 'seen', 'her'],
 ['had', 'served', 'him', 'so', 'well'],
 ['that', 'they', 'will', 'be', 'successful']

In [329]:
val_res = generate_modified_set(val_storage)
val_res

,modified_sentence,modified_word_idxs
0,"[that, might, bbmee, interpreted, as]",2
1,"[made, a, lobt, of, sacrifices]",2
2,"[plant, biological, ad, molecular, processes]",2
3,"[no, reason, wmvy, they, should]",2
4,"[and, he, ofld, her, he]",2
...,...,...
104422,"[what, else, a, we, do]",2
104423,"[too, much, emphass, is, placed]",2
104424,"[the, purpose, oc, having, a]",2
104425,"[gazing, up, xt, the, stars]",2


In [330]:
val_res['original_sent'] = val_storage
val_res

,modified_sentence,modified_word_idxs,original_sent
0,"[that, might, bbmee, interpreted, as]",2,"[that, might, be, interpreted, as]"
1,"[made, a, lobt, of, sacrifices]",2,"[made, a, lot, of, sacrifices]"
2,"[plant, biological, ad, molecular, processes]",2,"[plant, biological, and, molecular, processes]"
3,"[no, reason, wmvy, they, should]",2,"[no, reason, why, they, should]"
4,"[and, he, ofld, her, he]",2,"[and, he, told, her, he]"
...,...,...,...
104422,"[what, else, a, we, do]",2,"[what, else, can, we, do]"
104423,"[too, much, emphass, is, placed]",2,"[too, much, emphasis, is, placed]"
104424,"[the, purpose, oc, having, a]",2,"[the, purpose, of, having, a]"
104425,"[gazing, up, xt, the, stars]",2,"[gazing, up, at, the, stars]"


In [331]:
val_res.loc[:, 'modified_word_idxs'].unique()

array([2, 'do', 'up', 'to', 'be', 'me', 'at', 'on', 'we', 'as', 'of',
       'us', 'it', 'is', 'go', 'if', 'i', 'he', 'or', 'na', 'so', 'tv',
       'a', 'es', 'an', 'f'], dtype=object)

In [332]:
val_res.loc[(val_res.loc[:, 'modified_word_idxs'] != 1) & (val_res.loc[:, 'modified_word_idxs'] != 2) & (val_res.loc[:, 'modified_word_idxs'] != 3), :]

,modified_sentence,modified_word_idxs,original_sent
15,to,do,"[to, do, is, to, write]"
206,showed,up,"[showed, up, at, my, door]"
211,ways,to,"[ways, to, do, it, is]"
253,going,to,"[going, to, go, do, that]"
481,to,be,"[to, be, in, an, area]"
...,...,...,...
103631,up,to,"[up, to, go, to, work]"
103797,had,to,"[had, to, do, to, make]"
103983,going,to,"[going, to, be, a, quick]"
104041,came,up,"[came, up, to, me, and]"


In [333]:
# Remove such outliers
val_df = val_res.loc[(val_res.loc[:, 'modified_word_idxs'] == 1) | (val_res.loc[:, 'modified_word_idxs'] == 2) | (val_res.loc[:, 'modified_word_idxs'] == 3), :].reset_index(drop=True)
val_df

,modified_sentence,modified_word_idxs,original_sent
0,"[that, might, bbmee, interpreted, as]",2,"[that, might, be, interpreted, as]"
1,"[made, a, lobt, of, sacrifices]",2,"[made, a, lot, of, sacrifices]"
2,"[plant, biological, ad, molecular, processes]",2,"[plant, biological, and, molecular, processes]"
3,"[no, reason, wmvy, they, should]",2,"[no, reason, why, they, should]"
4,"[and, he, ofld, her, he]",2,"[and, he, told, her, he]"
...,...,...,...
103658,"[what, else, a, we, do]",2,"[what, else, can, we, do]"
103659,"[too, much, emphass, is, placed]",2,"[too, much, emphasis, is, placed]"
103660,"[the, purpose, oc, having, a]",2,"[the, purpose, of, having, a]"
103661,"[gazing, up, xt, the, stars]",2,"[gazing, up, at, the, stars]"


In [334]:
val_df.loc[:, 'modified_word_idxs'].unique()

array([2], dtype=object)

### Test Norvig's Corrector on validation set
Note that Norvig's Corrector does not take into the count neighboring words.

In [361]:
right_corrections_count = 0

for val_row in tqdm(val_df.iterrows(), total=val_df.shape[0]):
    if val_row[0] % 1000 == 0:
        print(f"Elements passed: {val_row[0]}. Accuracy: {(right_corrections_count / val_df.shape[0]):.4f}")
        print(f"Current correction result: {norvig_corrector.correction(val_row[1].loc['modified_sentence'][word_idx])}. Word: {val_row[1].loc['original_sent'][word_idx]}")
        
    word_idx = val_row[1].loc['modified_word_idxs']
    if val_row[1].loc['original_sent'][word_idx] == norvig_corrector.correction(val_row[1].loc['modified_sentence'][word_idx]):
        right_corrections_count += 1
    
print(f"Norvig Corrector accuracy of validation set: {(right_corrections_count / val_df.shape[0]):.4f}")

  0%|          | 0/103663 [00:00<?, ?it/s]

Elements passed: 0. Accuracy: 0.0000
Current correction result: bee. Word: be
Elements passed: 1000. Accuracy: 0.0053
Current correction result: a. Word: i
Elements passed: 2000. Accuracy: 0.0101
Current correction result: similar. Word: similar
Elements passed: 3000. Accuracy: 0.0152
Current correction result: have. Word: have
Elements passed: 4000. Accuracy: 0.0204
Current correction result: a. Word: a
Elements passed: 5000. Accuracy: 0.0255
Current correction result: same. Word: seem
Elements passed: 6000. Accuracy: 0.0306
Current correction result: to. Word: up
Elements passed: 7000. Accuracy: 0.0358
Current correction result: farm. Word: fat
Elements passed: 8000. Accuracy: 0.0409
Current correction result: that. Word: that
Elements passed: 9000. Accuracy: 0.0461
Current correction result: mm. Word: form
Elements passed: 10000. Accuracy: 0.0513
Current correction result: earlier. Word: earlier
Elements passed: 11000. Accuracy: 0.0562
Current correction result: needs. Word: needs
E

As you can see we managed to achieve ~53% accuracy on validation set using simplest Norvig's Corrector model implementation.

However, right now there are 2 unsolved problems:
- Accuracy can be improved;
- Model does not use other words in given sentences.

### Improve the basic implementation of Norvig's Corrector by adding the context usign N-Gram models

#### This section will be splitted into 2 parts:
- `NGramModel` class implementation;
- `NGramNorvigCorrector` class implementation.

In [335]:
class NGramModel():

    def __init__(self, ngram_sent: list[list[str]], n: int = 3) -> None:
        """
        Initialize the N-Gram language model.

        Parameters
        ----------
        ngram_sent: list[list[str]]
            List that contains sentences of fivegrams.
        n: int = 3
            The amount of words contained in each N-Gram.
        """

        self.n = n
        self.ngram_count = defaultdict(Counter)
        self.context_count = defaultdict(int)

        for ngram in ngram_sent:
            for i in range(len(ngram) - n + 1):
                cur_context = tuple(ngram[i:i + n - 1])
                word = ngram[i + n - 1]
                self.ngram_count[cur_context][word] += 1
                self.context_count[cur_context] += 1

    def P(self, word: str, context: defaultdict) -> float:
        """
        Calculate the probability of a word given its context.
        """

        context = tuple(context)
        if context in self.ngram_count:
            return self.ngram_count[context][word] / self.context_count[context]
        else:
            return 0.0

In [336]:
class NGramNorvigCorrector():

    def __init__(self, train_storage_dict: defaultdict, ngram_model: NGramModel) -> None:
        """
        Initialize the NorvigCorrector with N-Gram model.
        
        Parameters
        ----------
        train_dict: Counter
            Counter object that contains the amount of each word that is encountered in training text.
        ngram_model: NGramModel
            An instance of the N-Gram language model.
        """
        
        self.train_dict = train_storage_dict
        self.N = sum(train_storage_dict.values())
        self.ngram_model = ngram_model

    def P(self, word: str, N=None) -> float:
        """
        Calculate probability of given word.
        """

        if N is None:
            N = self.N
        return self.train_dict[word] / N

    def correction(self, word: str, context: defaultdict) -> str:
        """
        Find most probable spelling correction for given word, considering the context.
        """

        candidates = self.candidates(word)
        
        if not candidates:
            return word
        
        candidate_proba = []
        for candidate in candidates:
            word_proba = self.P(candidate)
            context_proba = self.ngram_model.P(candidate, context)
            #combined_proba = word_proba * context_proba
            # Try to add probabilitites instead of multiplying
            combined_proba = word_proba + context_proba
            candidate_proba.append((candidate, combined_proba))
        
        return max(candidate_proba, key=lambda x: x[1])[0]
    
    def candidates(self, word: str) -> set:
        """
        Generate possible spelling corrections for given word.
        """

        return (self.known([word]) or self.known(self.edits1(word))
                or self.known(self.edits2(word)) or [word])
    
    def known(self, words: str | list[str]) -> set:
        """
        Find the subset of words that appear in the dictionary of `NorvigCorrector.train_words`.
        """

        return set(w for w in words if w in self.train_dict.keys())
    
    def edits1(self, word: str) -> set:
        """
        Generate all possible edits that are one edit away from given word.
        """

        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)
    
    def edits2(self, word: str) -> set:
        """
        Generate all possible edits that are two edits away from given word.
        """
        
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

In [337]:
train_storage_dict = Counter(word for ngram in train_storage for word in ngram)

In [338]:
ngram_model = NGramModel(train_storage)

In [339]:
ngram_norvig_corrector = NGramNorvigCorrector(train_storage_dict, ngram_model)

In [349]:
test_example = random.choice(train_storage)
modified_test_example = modify_list(test_example)
word_number = modified_test_example[1]
print(f"Original test example: {test_example}.\n Modified test example: {modified_test_example[0]}.")

Original test example: ['do', "n't", 'understand', 'where', 'the'].
 Modified test example: ['do', "n't", 'unpderstand', 'where', 'the'].


In [350]:
modified_test_example[0][:word_number]

['do', "n't"]

In [351]:
ngram_norvig_corrector.correction(modified_test_example[0][word_number], modified_test_example[0][:word_number])

'understand'

In [352]:
modified_test_example[0][word_number]

'unpderstand'

### Test the upgraded Norvig's Corrector on validation dataset

In [353]:
right_corrections_count = 0

for val_row in tqdm(val_df.iterrows(), total=val_df.shape[0]):
    word_idx = val_row[1].loc['modified_word_idxs']
    modified_sent = val_row[1].loc['modified_sentence']
    correction = ngram_norvig_corrector.correction(modified_sent[word_idx], modified_sent[:word_idx])# + modified_sent[word_idx + 1:])

    if val_row[0] % 1000 == 0:
        print(f"Elements passed: {val_row[0]}. Accuracy: {(right_corrections_count / val_df.shape[0]):.4f}")
        print(f"Current correction result: {correction}. Word: {val_row[1].loc['original_sent'][word_idx]}")
        
    if val_row[1].loc['original_sent'][word_idx] == correction:
        right_corrections_count += 1
    
print(f"Upgraded Norvig Corrector accuracy of validation set: {(right_corrections_count / val_df.shape[0]):.4f}")

  0%|          | 0/103663 [00:00<?, ?it/s]

Elements passed: 0. Accuracy: 0.0000
Current correction result: bee. Word: be
Elements passed: 1000. Accuracy: 0.0058
Current correction result: i. Word: i
Elements passed: 2000. Accuracy: 0.0113
Current correction result: similar. Word: similar
Elements passed: 3000. Accuracy: 0.0171
Current correction result: have. Word: have
Elements passed: 4000. Accuracy: 0.0228
Current correction result: a. Word: a
Elements passed: 5000. Accuracy: 0.0286
Current correction result: seem. Word: seem
Elements passed: 6000. Accuracy: 0.0343
Current correction result: up. Word: up
Elements passed: 7000. Accuracy: 0.0400
Current correction result: farm. Word: fat
Elements passed: 8000. Accuracy: 0.0457
Current correction result: that. Word: that
Elements passed: 9000. Accuracy: 0.0515
Current correction result: mm. Word: form
Elements passed: 10000. Accuracy: 0.0573
Current correction result: earlier. Word: earlier
Elements passed: 11000. Accuracy: 0.0629
Current correction result: needs. Word: needs
E

### Conclusion & improvement thoughts
As we can see, adding the N-Gram model and summing total probabilitites for each word improves accuracy a bit.
From my point of view, in the future Norvig Correctors with N-Gram models implementations the following modifications can be applied:
- Try to add weights to `word_proba` and `context_proba`;
- Try to add weights to all error types;
- Instead of adding weights to probabilities we can think of different formula for the resulting probability (Not a simple summation);
- Many more... Imagination is the main limitation. And computing power + storage.

#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)